API 키 로드
`.env` 에 설정되어 있어야 합니다.
```
OPENAI_API_KEY=sk-proj-******** # Your Key
```

In [1]:
from dotenv import load_dotenv

# API 키 정보 로드
load_dotenv()

True

LangSmith 추적을 설정합니다.(선택)

In [2]:
# LangSmith 추적을 설정합니다. https://smith.langchain.com
# !pip install -qU langchain-teddynote
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("AI-SEARCH")

LangSmith 추적을 시작합니다.
[프로젝트명]
AI-SEARCH


In [3]:
# 라이브러리 선언
import sqlite3
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain_openai import ChatOpenAI
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import PromptTemplate
import re
import time

보통 Vector Store 를 생성한 임베딩과 동일한 임베딩을 사용합니다.

In [4]:
# 1. 임베딩(Embedding) 생성

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")  # 1536 차원

# dimensions 파라미터 설정 (원하는 차원 수로 축소)
# embeddings = OpenAIEmbeddings(
#     model="text-embedding-3-small",
#     dimensions=512  # 512 차원으로 축소 (최대 1536까지 가능)
# )

사용자의 의도를 분석하여 필터를 생성합니다.

#### 텍스트형 & 숫자+텍스트 혼합형 금액 치환

In [5]:
PRICE_UNIT = {
    "일": 1,
    "이": 2,
    "삼": 3,
    "사": 4,
    "오": 5,
    "육": 6,
    "칠": 7,
    "팔": 8,
    "구": 9,
    "십": 10,
    "백": 100,
    "천": 1000,
    "만": 10000,
    "십만": 100000,
    "백만": 1000000
}

def price_info(price_match):
    base_price = 1
    text_price = 0
    # 숫자형 금액
    if (price_match.group(1)):
        base_price *= int(price_match.group(1))
    # 문자형 금액
    first_unit = base_price
    if (price_match.group(2) or price_match.group(3)):
        if (price_match.group(2)):
            first_unit *= PRICE_UNIT[price_match.group(2)]
        if (price_match.group(3)):
            if (len(price_match.group(3)) > 2):
                first_unit *= PRICE_UNIT[price_match.group(3)[0]]
                if (str(first_unit)[0] == 1):
                    first_unit *= PRICE_UNIT[price_match.group(3)[1]]
                else:
                    first_unit += PRICE_UNIT[price_match.group(3)[1]] + PRICE_UNIT[price_match.group(3)[2]]
            elif (len(price_match.group(3)) > 1):
                first_unit *= PRICE_UNIT[price_match.group(3)[0]]
                if (str(first_unit)[0] == 1):
                    first_unit *= PRICE_UNIT[price_match.group(3)[1]]
                else:
                    first_unit += PRICE_UNIT[price_match.group(3)[1]]
            else:
                first_unit *= PRICE_UNIT[price_match.group(3)]
    text_price += first_unit

    second_unit = 0
    if (price_match.group(4)):
        second_unit = int(price_match.group(4))
    if (price_match.group(5) or price_match.group(6)):
        if (price_match.group(5)):
                second_unit = PRICE_UNIT[price_match.group(5)]
        if (price_match.group(6)):
            if (len(price_match.group(6)) > 1):
                second_unit *= PRICE_UNIT[price_match.group(6)[0]]
                text_price += second_unit
                text_price *= PRICE_UNIT[price_match.group(6)[1]]
            else:
                second_unit *= PRICE_UNIT[price_match.group(6)]
    text_price += second_unit
        

    third_unit = 0
    if (price_match.group(7)):
        third_unit = int(price_match.group(7))
    if (price_match.group(8) or price_match.group(9)):
        if (price_match.group(8)):
                third_unit = PRICE_UNIT[price_match.group(8)]
        if (price_match.group(9)):
            if (len(price_match.group(9)) > 1):
                third_unit *= PRICE_UNIT[price_match.group(9)[0]]
                text_price += third_unit
                text_price *= PRICE_UNIT[price_match.group(9)[1]]
            else:
                third_unit *= PRICE_UNIT[price_match.group(9)]
    text_price += third_unit

    fourth_unit = 0
    if (price_match.group(10)):
        fourth_unit = int(price_match.group(10))
    if (price_match.group(11)):
        if (price_match.group(11)):
            fourth_unit = PRICE_UNIT[price_match.group(11)]
    text_price += fourth_unit
    
    base_price = text_price * 10000
    return base_price

In [6]:
# 2. 의도 기반 필터링 함수 선언

def intent_based_filtering(query):
    """사용자 의도 분석을 통한 필터링"""

    filter_dict = {}

    # 가격 필터링을 위한 정규표현식 패턴
    # 1) N만원 이상 | N만원 부터
    price_pattern = r'(\d*)([일이삼사오육칠팔구]*)([십백천만]*)(\d*)([일이삼사오육칠팔구]*)([십백천만]*)(\d*)([일이삼사오육칠팔구]*)([십백천만]*)(\d*)([일이삼사오육칠팔구]*)만원\s*(이상|부터|초과)'
    price_match = re.search(price_pattern, query)
    
    if price_match:
        # SALE_PRC 키가 없으면 생성
        if "SALE_PRC" not in filter_dict:
            filter_dict["SALE_PRC"] = {}
        # 만원 단위를 원 단위로 변환 (예: 120만원 -> 1200000원)
        base_price = price_info(price_match)
        if (base_price > 0):
            filter_dict["SALE_PRC"]["$gte"] = base_price
    
    # 2) N만원 이하 | N만원 까지
    price_pattern_lte = r'(\d*)([일이삼사오육칠팔구]*)([십백천만]*)(\d*)([일이삼사오육칠팔구]*)([십백천만]*)(\d*)([일이삼사오육칠팔구]*)([십백천만]*)(\d*)([일이삼사오육칠팔구]*)만원\s*(이하|까지)'
    price_match_lte = re.search(price_pattern_lte, query)
    
    if price_match_lte:
        if "SALE_PRC" not in filter_dict:
            filter_dict["SALE_PRC"] = {}
        base_price = price_info(price_match_lte)
        if (base_price > 0):
            filter_dict["SALE_PRC"]["$lte"] = base_price
    # 3) N만원 대
    price_pattern_range = r'(\d*)([일이삼사오육칠팔구]*)([십백천만]*)(\d*)([일이삼사오육칠팔구]*)([십백천만]*)(\d*)([일이삼사오육칠팔구]*)([십백천만]*)(\d*)([일이삼사오육칠팔구]*)만원\s*대'
    price_match_range = re.search(price_pattern_range, query)
    
    if price_match_range:
        if "SALE_PRC" not in filter_dict:
            filter_dict["SALE_PRC"] = {}
        base_price = price_info(price_match_range)
        if (base_price > 0):
            filter_dict["SALE_PRC"]["$gte"] = base_price
            # (N만원 + 최대 자릿수)까지
            # 240만원: 240만원 ~ 340만원
            filter_dict["SALE_PRC"]["$lt"] = base_price + (10 ** (len(str(base_price)) - 1))

    return filter_dict

### Vectorestore 검색 1 - 검색기만을 통한 검색
- 속도가 빠릅니다
- 검색어와 필터를 통해서만 검색결과를 조정할 수 있습니다.

In [7]:

# 3. Vectorstore 검색 1 - 검색기만을 통한 검색

# DB 경로 설정
persist_directory = "../.db/faiss"

vectorstore = FAISS.load_local(
    persist_directory, 
    embeddings, 
    allow_dangerous_deserialization=True)

# 검색할 쿼리 설정 (원하는 검색어로 변경 가능)
query = "현대카드 할인하는 LG전자 TV"  # 예시 쿼리: 원하는 검색어로 변경하세요

intent_start = time.time()

# 필터링 옵션 입력 받기
filter_dict = intent_based_filtering(query)
# filter_dict = {"GOODS_NO": '0031055945'}
# filter_dict = {
#     "SALE_PRC": {
#         "$gte": 1000000,  # 최소 가격
#         "$lte": 2000000   # 최대 가격
#     }
# }

print(f"filter_dict: {filter_dict}")

# 쿼리와 유사한 상품 5개 검색
results_with_score = vectorstore.similarity_search_with_score(
    query=query, 
    filter = filter_dict, # 메타데이터 필터 적용
    k=5  # 반환할 결과 수
)

print(f"⚡ intent_baesd 소요 시간: {time.time() - intent_start:.2f}초")

# 결과 출력
print(f"'{query}'와 유사한 상품 검색 결과 (최대 5개):")
print("-" * 60)

if results_with_score:
    for i, (doc, score) in enumerate(results_with_score, 1):
        print(f"[결과 {i}] 유사도: {score}") # 낮을수록 더 유사함
        print(f"{doc.page_content}")
        # print(f"메타데이터: {doc.metadata}")  # 모든 메타데이터 출력
        try:
            print(f"- 상품상태: {doc.metadata.get('GOODS_STAT_SCT_NM', '정보 없음')}")
            print(f"- 브랜드명: {doc.metadata.get('BRND_NM', '정보 없음')}")
            print(f"- 상품명: {doc.metadata.get('GOODS_NM', '정보 없음')}")
            print(f"- 품목: {doc.metadata.get('ARTC_NM', '정보 없음')}")
            print(f"- 카테고리: {doc.metadata.get('CATEGORY_NM', '정보 없음')}")
            print(f"- 판매가: {format(int(doc.metadata.get('SALE_PRC', 0)), ',')}원")
            print(f"- 할인가: {format(int(doc.metadata.get('DSCNT_SALE_PRC', 0)), ',')}원")
            print(f"- 최대혜택가: {format(int(doc.metadata.get('MAX_BENEFIT_PRICE', 0)), ',')}원")
            print(f"- 카드할인율: {doc.metadata.get('CARD_DC_RATE', '0')}%")
            print(f"- 할인카드: {doc.metadata.get('CARD_DC_NAME_LIST', '정보 없음')}")
            print(f"- 주요 특징 및 기능:")
            feature_values = doc.metadata['OPT_VAL_DESC'].split(',')
            feature_titles = doc.metadata['OPT_DISP_NM'].split(',')
            for i, (title, value) in enumerate(zip(feature_titles, feature_values)):
                print(f"  - {title} : {value}")
                if i < 3: break
            print(f"🔗 상품보러가기 : https://www.e-himart.co.kr/app/goods/goodsDetail?goodsNo={doc.metadata.get('GOODS_NO', '정보 없음')}")
        except Exception as e:
            print(f"메타데이터 처리 중 오류 발생: {str(e)}")
        print("-" * 60)
else:
    print("검색 결과가 없습니다.")
    print("다른 검색어로 다시 시도해보시거나, 검색어를 더 구체적으로 입력해주세요.")


filter_dict: {}
⚡ intent_baesd 소요 시간: 3.50초
'현대카드 할인하는 LG전자 TV'와 유사한 상품 검색 결과 (최대 5개):
------------------------------------------------------------
[결과 1] 유사도: 0.7212498188018799
LG전자 브랜드의 LCD 일반 제품이며, 상품명 LG PC 모니터  27BA400.BKR. 모델명: 27BA400.BKR. 카테고리: 컴퓨터·노트북. 가격 정보: 판매가 169,000원, 현대카드 카드 사용 시 6% 할인되어 최대 158,860원. 2025년 3월에 출시되었으며 최근 한 달 동안 14개 판매됨.
- 상품상태: 정상상품
- 브랜드명: LG전자
- 상품명: LG PC 모니터 (68.6) 27BA400.BKR
- 품목: LCD 일반
- 카테고리: 정보 없음
- 판매가: 189,000원
- 할인가: 169,000원
- 최대혜택가: 158,860원
- 카드할인율: 6%
- 할인카드: 현대카드,삼성카드
- 주요 특징 및 기능:
  -  : 
🔗 상품보러가기 : https://www.e-himart.co.kr/app/goods/goodsDetail?goodsNo=0041276524
------------------------------------------------------------
[결과 2] 유사도: 0.812940239906311
LG전자 브랜드의 상품명 120cm OLED TV OLED48A1ENA . 모델명: OLED48A1ENA.AKRG. 카테고리: TV·영상가전. 가격 정보: 판매가 1,590,000원, 현대카드 카드 사용 시 6% 할인되어 최대 1,494,600원. 해시태그: #LG전자#TV#OLED#UHD#48인치#4KUHD#스마트. 2021년 11월에 출시되었으며 최근 한 달 동안 0개 판매됨.
- 상품상태: 정상상품
- 브랜드명: LG전자
- 상품명: 120cm OLED TV OLED48A1ENA (설치유형 선택가능)

### Vectorestore 검색 2 - LLM 과 검색기를 체인으로 생성하여 검색 
- 속도가 느립니다.
- 검색어와 필터를 뿐 만아니라 프롬프트로도 검색결과를 조정할 수 있습니다.
- 친절하고 감성적인 답변이 가능합니다.

In [143]:
# 1. 프롬프트를 생성합니다.
prompt1 = PromptTemplate.from_template(
    """You are a product search assistant for an electronics e-commerce platform. 
Use the following pieces of retrieved context to answer the question. 
Search up to 10 products.
Kindly explain why you've been searching for the product.
And be sure to include the following meta information:
상품명
- 상품상태
- 브랜드명
- 판매가
- 할인가
- 최대혜택가
- 할인카드
- 주요 특징 및 기능
  - 
  -
  -
🔗 상품보러가기 : https://www.e-himart.co.kr/app/goods/goodsDetail?goodsNo=상품번호
Answer in Korean.

#Context: 
{context}

#Question:
{question}

#Answer:"""
)


In [144]:
# 1. 프롬프트를 생성합니다.
prompt2 = PromptTemplate.from_template(
    """당신은 전자제품 전문 쇼핑몰의 상품 검색 도우미입니다.
아래 제공된 상품 정보를 기반으로 고객의 질문에 답변해주세요.

[답변 작성 지침]
1. 최대 10개의 상품을 검색하여 추천해주세요.
2. 고객이 이 상품들을 찾게 된 맥락과 추천 이유를 먼저 설명해주세요.
3. 상품 정보는 다음 순서로 정확하게 표기해주세요:
   
상품명
- 상품상태
- 브랜드명
- 판매가
- 할인가
- 최대혜택가
- 할인카드
- 주요 특징 및 기능
  - 
  -
  -
🔗 상품보러가기 : https://www.e-himart.co.kr/app/goods/goodsDetail?goodsNo=상품번호

4. 가격 정보는 모두 쉼표(,)를 포함한 숫자 형식으로 표기해주세요.
5. 상품 간 구분은 위의 구분선을 사용해주세요.
6. 마지막에는 전체 상품에 대한 간단한 총평을 덧붙여주세요.

[참고 정보]
{context}

[고객 질문]
{question}

[답변]
"""
)

In [145]:
# 2. LLM 과 체인생성

# 모델(LLM) 을 생성합니다.
llm = ChatOpenAI(model_name="gpt-4o", temperature=0)

# 의도 기반 필터링 함수 선언
def get_retriever_with_filter(query):
    filter_dict = intent_based_filtering(query)

    print(f"필터정보:{filter_dict}")

    return vectorstore.as_retriever(
        search_kwargs={
            "k": 5,
            "filter": filter_dict
        }
    )

# 체인(Chain) 생성
chain1 = (
    {"context": lambda x: get_retriever_with_filter(x).get_relevant_documents(x), 
     "question": RunnablePassthrough()}
    | prompt1
    | llm
    | StrOutputParser()
)

chain2 = (
    {"context": lambda x: get_retriever_with_filter(x).get_relevant_documents(x), 
     "question": RunnablePassthrough()}
    | prompt2
    | llm
    | StrOutputParser()
)

In [146]:
# 3. 체인 실행
# query = "200만원 이하 삼성 스마트 세탁기"  # 예시 쿼리: 원하는 검색어로 변경하세요
query = "현대카드 할인하는 LG전자 TV"  # 예시 쿼리: 원하는 검색어로 변경하세요

llm_start = time.time()

response = chain1.invoke(query)

print(f"🍖LLM prompt 1️⃣번 결과 =========================================")
print(f"⚡ llm_baesd 소요 시간: {time.time() - llm_start:.2f}초")
print(response)

필터정보:{}
🍖LLM prompt 1️⃣번 결과 =========================================
⚡ llm_baesd 소요 시간: 12.86초
현대카드 할인이 가능한 LG전자 TV를 찾기 위해 검색을 진행했습니다. 현대카드 할인이 적용되는 제품은 다음과 같습니다:

1. **상품명**: 120cm LG전자 4K UHD TV OLED48C4ENA (설치유형 선택가능)
   - **상품상태**: 정상상품
   - **브랜드명**: LG전자
   - **판매가**: 1,490,000원
   - **할인가**: 1,490,000원
   - **최대혜택가**: 1,341,000원
   - **할인카드**: 롯데카드, 현대카드 등
   - **주요 특징 및 기능**:
     - 4K UHD 해상도
     - OLED 디스플레이
     - 설치유형 선택 가능
   - 🔗 [상품보러가기](https://www.e-himart.co.kr/app/goods/goodsDetail?goodsNo=0035753360)

2. **상품명**: 194cm 4K UHD evo OLED77C3QNA [설치유형 선택가능]
   - **상품상태**: 정상상품
   - **브랜드명**: LG전자
   - **판매가**: 6,600,000원
   - **할인가**: 6,600,000원
   - **최대혜택가**: 6,450,000원
   - **할인카드**: 롯데카드, 현대카드 등
   - **주요 특징 및 기능**:
     - 4K UHD 해상도
     - evo OLED 디스플레이
     - 설치유형 선택 가능
   - 🔗 [상품보러가기](https://www.e-himart.co.kr/app/goods/goodsDetail?goodsNo=0021346242)

3. **상품명**: 120cm OLED 4K TV OLED48C3ENA [설치유형 선택가능]
   - **상품상태**: 정상상품
   - **브랜드명**: LG전자
   - **판매가**: 1,

In [147]:
# 3. 체인 실행
llm_start = time.time()

response = chain2.invoke(query)

print(f"🤸‍♂️LLM prompt 2️⃣번 결과 =========================================")
print(f"⚡ llm_baesd 소요 시간: {time.time() - llm_start:.2f}초")
print(response)

필터정보:{}
🤸‍♂️LLM prompt 2️⃣번 결과 =========================================
⚡ llm_baesd 소요 시간: 12.69초
고객님께서 현대카드 할인을 받을 수 있는 LG전자 TV를 찾고 계시군요. 현대카드로 구매 시 할인 혜택을 받을 수 있는 다양한 LG전자 TV를 추천해드리겠습니다. 이 TV들은 최신 기술을 탑재하고 있어 뛰어난 화질과 성능을 자랑합니다. 아래의 상품들을 확인해보세요.

---

120cm LG전자 4K UHD TV OLED48C4ENA (설치유형 선택가능)
- 상품상태: 정상상품
- 브랜드명: LG전자
- 판매가: 1,490,000원
- 할인가: 1,490,000원
- 최대혜택가: 1,341,000원
- 할인카드: 롯데카드
- 주요 특징 및 기능:
  - 4K UHD 해상도
  - OLED 디스플레이
  - 다양한 설치 유형 선택 가능
🔗 상품보러가기 : https://www.e-himart.co.kr/app/goods/goodsDetail?goodsNo=0035753360

---

194cm 4K UHD evo OLED77C3QNA [설치유형 선택가능]
- 상품상태: 정상상품
- 브랜드명: LG전자
- 판매가: 6,600,000원
- 할인가: 6,600,000원
- 최대혜택가: 6,450,000원
- 할인카드: 롯데카드
- 주요 특징 및 기능:
  - 초대형 4K UHD 해상도
  - evo OLED 기술
  - 다양한 설치 유형 선택 가능
🔗 상품보러가기 : https://www.e-himart.co.kr/app/goods/goodsDetail?goodsNo=0021346242

---

120cm OLED 4K TV OLED48C3ENA [설치유형 선택가능]
- 상품상태: 정상상품
- 브랜드명: LG전자
- 판매가: 1,726,200원
- 할인가: 1,726,200원
- 최대혜택가: 1,639,890원
- 할인카드: 롯데카드
- 주요 특징 및 기능:
  - 4K 해상도
  - OL